In [1]:
import sys
from rocket_functions import generate_kernels, apply_kernels
from sklearn.linear_model import SGDClassifier
import numpy as np
import pickle
import math
from sklearn.metrics import classification_report
import shap
import time
import csv
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
def normalization(X_training, X_test):   
    dataref = np.concatenate((X_training, X_test), axis = 0)
    dataref_plain = dataref.reshape(dataref.shape[0]*dataref.shape[1], 1)
    X_training_plain = X_training.reshape(X_training.shape[0]*X_training.shape[1], 1)
    X_test_plain = X_test.reshape(X_test.shape[0]*X_test.shape[1], 1)
    scaler = MinMaxScaler()
    scaler.fit(dataref_plain)
    X_training_norm = scaler.transform(X_training_plain)
    X_test_norm = scaler.transform(X_test_plain)
    X_training_norm = X_training_norm.reshape(X_training.shape[0], X_training.shape[1])
    X_test_norm = X_test_norm.reshape(X_test.shape[0], X_test.shape[1])
    return X_training_norm, X_test_norm


def normalization_estesa(X_training, X_test):   
    dataref = np.concatenate((X_training, X_test), axis = 0)
    dataref_plain = dataref.reshape(dataref.shape[0]*dataref.shape[1], 1)
    X_training_plain = X_training.reshape(X_training.shape[0]*X_training.shape[1], 1)
    X_test_plain = X_test.reshape(X_test.shape[0]*X_test.shape[1], 1)
    massimo = max(dataref_plain)
    minimo = min(dataref_plain)
    X_training_norm_plain = (X_training_plain - minimo)/(massimo - minimo)
    X_training_norm = X_training_norm_plain.reshape(X_training.shape[0], X_training.shape[1])
    return X_training_norm, massimo, minimo


def normalization_shap(x):
    x_norm = (x - minimo)/(massimo - minimo)
    return x_norm


def read_dataset_modified(dataset_folder, rischio, scaling, overunder, perc_min = 40, perc_magg = 60):
    if overunder != '' and scaling != '':
        add_str_ytrain = add_str_xtrain  = '_' + scaling +'_' + overunder + '_' + str(perc_min) + '_' + str(perc_magg)
        add_str_xtest  = '_' + scaling
        add_str_ytest = ''
    elif overunder != '' and scaling == '' :
        add_str_ytrain = add_str_xtrain = '_' + overunder + '_' + str(perc_min) + '_' + str(perc_magg)
        add_str_ytest = add_str_xtest = ''
    elif overunder == '' and scaling != '' :
        add_str_ytrain = add_str_ytest = ''
        add_str_xtrain = add_str_xtest = '_' + scaling
    elif overunder == '' and scaling == '' :
        add_str_ytrain = add_str_ytest = add_str_xtrain = add_str_xtest = ''
    else:
        print('PROBLEMA', overunder, scaling )
        return
    with open(dataset_folder + '/training' + rischio + add_str_xtrain + '.p', 'rb') as f:
        x_train = pickle.load(f)
        X_training = x_train.astype(np.float64)
    with open(dataset_folder + '/label_training' + rischio + add_str_ytrain + '.p', 'rb') as f:
        y_train = pickle.load(f)
        Y_training = y_train.astype(np.int32)
    with open(dataset_folder + '/test' + rischio + add_str_xtest + '.p', 'rb') as f:
        x_test = pickle.load(f)
        X_test = x_test.astype(np.float64)
    with open(dataset_folder + '/label_test' + rischio + add_str_ytest + '.p', 'rb') as f:
        y_test = pickle.load(f)
        Y_test = y_test.astype(np.int32)
    return X_training, Y_training, X_test, Y_test

def dataset_name_f(strrischio, overunder, normst):
    str_overunder_dataset = ''
    str_norm_dataset = ''
    if normst != '':
        str_norm_dataset = '_' + normst
    if overunder != '':
        if strrischio == '2' or strrischio == '5':
            str_overunder_dataset = '_' + overunder +'_40_60'  
        else:
            str_overunder_dataset = '_' + overunder +'_50_50'
    dataset_name = 'rischio' + strrischio + str_norm_dataset + str_overunder_dataset
    return dataset_name


def choose_kernels_perc(rischio):
    if rischio == '2' or rischio == '3':
        f = open('/home/amatteoli/rocket-prove/MODELLI_SCELTI/kernels_2_3.pckl', 'rb')
        kernels = pickle.load(f)
        f.close()   
    elif rischio == '4':
        f = open('/home/amatteoli/rocket-prove/MODELLI_SCELTI/kernels_4.pckl', 'rb')
        kernels = pickle.load(f)
        f.close()
    elif rischio == '5':
        f = open('/home/amatteoli/rocket-prove/MODELLI_SCELTI/kernels_5.pckl', 'rb')
        kernels = pickle.load(f)
        f.close()
    else:
        print('rischio inesistente')
    
    if rischio == '2' or rischio == '5':
        perc_min = 40
        perc_mag = 60
    else:
        perc_mag = perc_min = 50
    return kernels, perc_min, perc_mag

def pred(x):
    x_transform = apply_kernels(x, kernels)
    x_transform = normalization_shap(x_transform)
    predictions = classifier.predict(x_transform)
    return predictions

def pred_prob(x):
    x_transform = apply_kernels(x, kernels)
    x_transform = normalization_shap(x_transform)
    prob_pred = classifier.predict_proba(x_transform)
    return prob_pred

def pred_prob_RC(x):
    x_transform = apply_kernels(x, kernels)
    x_transform = normalization_shap(x_transform)
    d = classifier.decision_function(x_transform)
    probs = np.array([np.exp(-d) / (1 + np.exp(-d)), np.exp(d) / (1 + np.exp(d))]).flatten()
    return(probs)


"def shannon_entropy(X_training, X_test, pad):\n    #costruisco un dizionario con la shannon entropy totale per ogni luogo\n    shannon_tot = {}\n    #metto l'entropia del padding a 0\n    shannon_tot[pad] = 0\n    dataset = np.concatenate((X_training, X_test))\n    for trajectory in dataset:\n        #per ogni traiettoria nel training (o dovrei utilizzare anche il test?), posso utilizzare i \n        #training con over under? cioè se l'errore è piccolo\n        current_traj = list(filter(lambda a: a != pad, trajectory))\n        tot_locations = previous_len = len(current_traj)\n        for location in set(current_traj):\n            #elimino la locazione corrente e calcolo la nuova lunghezza per l'utente\n            current_traj = list(filter(lambda a: a != location, current_traj))\n            current_len = len(current_traj)\n            #calcolo la shannon entropy per quell'utente e quella traiettoria \n            prob = (previous_len - current_len)/tot_locations\n            curr

In [3]:
#construisco un dizionario che contiene il totale di volte che un luogo è stato visitato:
def dict_tot_visits(X_training, X_test, pad):
    tot_visits = {}
    dataset = np.concatenate((X_training, X_test))
    for trajectory in dataset:
        for location in trajectory:
            if location != pad:
                tot_visits[location] = tot_visits.get(location, 0) + 1
    return tot_visits
                
    
def shannon_entropy(X_training, X_test,  pad):
    #costruisco un dizionario con la shannon entropy totale per ogni luogo 
    shannon_tot = {}
    shannon_tot[pad] = float('Inf')
    tot_visits = dict_tot_visits(X_training, X_test, pad)
    #metto l'entropia del padding a Inf
    dataset = np.concatenate((X_training, X_test))
    for trajectory in dataset:
        #per ogni traiettoria 
        #tolgo il padding
        current_traj = list(filter(lambda a: a != pad, trajectory))
        for location in set(current_traj):
            occurrences = current_traj.count(location)
            #calcolo la shannon entropy per quell'utente in quella posizione
            prob = occurrences/tot_visits[location]
            current_shannon = - prob * math.log(prob,2)  
            #la sommo alle altre
            shannon_tot[location] = shannon_tot.get(location, 0) + current_shannon
    return(shannon_tot)


def transformer_shannon_tot_coord(shannon_tot, scaling_to_int_ref):
    shannon_tot_int = {}
    for key in shannon_tot:
        shannon_tot_int[scaling_to_int_ref[key]] = shannon_tot[key]
    return shannon_tot_int 
        

#maschera per il singolo utente traj x
def funct_mask(x, pad, shannon_tot_adapted):
    x_mask = [0] * len(x)
    traj_len = len(list(filter(lambda a: a != pad, x)))
    #individuo il numero che da 30% locazioni
    n_feat = math.ceil((30 * traj_len)/100)
    enum = list(enumerate(x))
    #lista degli indici delle posizioni da mettere a uno nella maschera
    ones = [x for x,y in sorted(enum, key = lambda e: shannon_tot_adapted[e[1]])[:n_feat]]
    for index in ones:
        x_mask[index] = 1
    return x_mask


def custom_masker(mask, x):
    return (x * mask).reshape(1,len(x))

In [4]:
def run_kernels_pre_shap(X_training, X_test, kernels):
    inizio = time.time()
    X_training_transform = apply_kernels(X_training, kernels)
    fine = time.time()
    print('Running kernels su train:' , fine - inizio)
    inizio = time.time()
    X_test_transform = apply_kernels(X_test, kernels)
    fine = time.time()
    print('Running kernels su test:' , fine - inizio)
    _, massimo, minimo = normalization_estesa(X_training_transform, X_test_transform)
    return(massimo, minimo)

In [5]:
overunder = '' #'', 'randunder', 'IHTunder', 'randover', 'smote', 
scaling = 'norm' #'', 'norm', 'std'
rischio = '4'
classifier_type = 'SDG'

dataset = dataset_name_f(rischio, overunder, scaling)

dataset_folder = '/datasets_tesi/pad_incima/rischio' + rischio
models_folder = '/rocket-prove/MODELLI_SCELTI/voronoi/'
classifier_name = classifier_type + '_' + dataset + '_modello.sav'
shap_folder = '/SHAP/'
tempo_shapley = 'shapley_values_time.txt'
shapley_file = 'shap_values_rocket_' + classifier_type + '_' + dataset

kernels, perc_min, perc_mag = choose_kernels_perc(rischio)

X_training, Y_training, X_test, Y_test = read_dataset_modified(dataset_folder, rischio, scaling,
                                                                    overunder, perc_min, perc_mag)

In [6]:
pad = X_training[0][0]

In [7]:
with open(shap_folder + 'shannon_tot_int_general2.p', 'rb') as handle:
    shannon_tot_int = pickle.load(handle)

In [9]:
path = shap_folder + 'coord_mappings/'
risk_dataset = 'rischio' + rischio


if scaling != '':
    with open(path + 'map_' + scaling + '_to_int_' + risk_dataset + '.p', 'rb') as f:
            norm_to_int = pickle.load(f)

    shannon_tot_adapted = {}

    for key_norm, key_int in norm_to_int.items():
        shannon_tot_adapted[key_norm] = shannon_tot_int[key_int]

    del shannon_tot_int
    del norm_to_int
else:
    shannon_tot_adapted = shannon_tot_int

In [10]:
classifier = pickle.load(open(models_folder + classifier_name, 'rb'))
massimo, minimo = run_kernels_pre_shap(X_training, X_test, kernels)

Running kernels su train: 71.63004660606384
Running kernels su test: 18.072691679000854


In [ ]:
inizio = time.time()
for index,x in enumerate(X_test):
    mask = funct_mask(x, pad, shannon_tot_adapted)
    explainer = shap.Explainer(pred_prob, custom_masker)
    shap_values = explainer.shap_values(np.array([x]))
    shap_tuple = (index,shap_values)
    with open(shap_folder + shapley_file + '.p', 'ab') as fp:
        pickle.dump(shap_tuple,fp)
    del explainer
    print(index)
fine = time.time()
tempo = fine - inizio
with open(shap_folder + tempo_shapley , 'a') as f:
    f.write(shapley_file + '  ' + str(tempo))

Permutation explainer: 2it [00:40, 40.38s/it]                                                                           


0


Permutation explainer: 2it [00:39, 39.80s/it]                                                                           


1
